In [1]:
from NBfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
from preprocessor import Preprocessor
from referencer import StateReferencer
from textwrap import TextWrapper
from sklearn.svm import SVC
from copy import copy

state_defaults = {
    'label':'New State',
    'model':'SVC',
    'preprocessor': Preprocessor(),
    'transitions':[],
    'slots':[],
    'triggers':[],
    'segmenter':None
}

model_factory = {
    'SVC':SVC    
}

model_yrotcaf = { v:k for (k,v) in model_factory.items() }



class State(StateReferencer):
    """The state objects will store data if they dont have a parent but refer to the parent if they do have a parent.
    The exception to this is the model which is always kept in this object."""
    
    model_factory = model_factory

    def __init__(self, label=None, parent=None, x=None, y=None, model=None, transitions=None, 
                 preprocessor=None, slots=None, triggers=None, segmenter=None):
        StateReferencer.__init__(self)
        self.ID = id(self)
        if parent and type(parent.state_data)!=dict:
            raise ValueError("Invalid container for state:", str(label))
        if parent and label in parent.state_data['label']:
            self.parent = None
            self.container=None
        else:
            self.parent = parent
            self.container=parent.state_data
        
        self.label=label
        self.x= x
        self.y= y
        self.model= model  or state_defaults['model']
        self.transitions= transitions or state_defaults['transitions']
        self.slots= slots or state_defaults['slots']
        self.triggers= triggers or state_defaults['triggers']
    
    def connect(self, target, label=None, **kwargs):
        if not label:
            if type(target)==str:
                label = str(self.label) + "_to_" + target
            elif type(target)==State:
                label = str(self.label) + "_to_" + str(target.label)
        transition=self.parent.transition(label, self, target, **kwargs)
        self.transitions=self.transitions+[transition]
        return transition
    
    def _make_copy(self, container=None):
        new = copy(self)
        new._copy_values_to_local(self)
        new.container=container
        if container:
            new._read_local_values()
        return new   
                
    def _copy_values_to_local(self, source):
        """Copy the values from a state object (possibly self) into the local references for this object.  
        I.E. regardless of if it is attached to a container or not, load to self._X, self._y, etc. instead of self.x"""
        (self._label, self._model, self._transitions, 
        self._slots, self._triggers, self._x, self._y) =\
        (source.label, source.model, source.transitions,
        source.slots, source.triggers, source.x, source.y)
        
    def _read_local_values(self):
        """If attached to a container, copy any local values to the container.  Otherwise do nothing."""
        if self.container:
            (self.label, self.model, self.transitions,
            self.slots, self.triggers, self.x, self.y) =\
            (self._label, self._model, self._transitions, 
            self._slots, self._triggers, self._x, self._y)
            
            del (self._label, self._model, self._transitions, 
            self._slots, self._triggers, self._x, self._y)
        else:
            pass
    
    def attach(self, container):
        self.container=container
        self.label=label
        self.x= self._x
        self.y= self._y
        self.model= self._model
        self.transitions= self._transitions
        self.slots= self._slots
        self.triggers= self._triggers
        return self
    
    def dettach(self):
        """Copy all of the information into local variables and remove the container reference."""
        
        (self._label, self._model, self._transitions, 
        self._slots, self._triggers, self._x, self._y) =\
        (self.label, self.model, self.transitions,
        self.slots, self.triggers, self.x, self.y)
        self.container=None
        return self
    
    def __str__(self):
        readable= f"State:{self.label}| ID:{id(self)}"
        for transition in self.transitions:
            readable+=TextWrapper(initial_indent='\n\t', subsequent_indent='\n\t').fill(str(transition))
        return readable
    
    def _ipython_display_(self):
        print(self.__str__())

importing Jupyter notebook from preprocessor.ipynb
importing Jupyter notebook from referencer.ipynb
